## Suggested solution for Exercise 4 (Overdetermined systems)

In [1]:
using HomotopyContinuation

Suppose that we want to solve a system $F\in(\mathbb{C}[x_1,\ldots,x_n])^m$ that is overdetermined, in the sense that $m>n$ (i.e., we have more polynomials than variables).

**Problem:** Solving the Davidenko ODEs relies on inverting the Jacobian. In particular, the Jacobian needs to be a square matrix, which only happens if the number of equations equals the number of variables.

**Strategy:** Turn $F\in\mathbb{C}[x_1,\ldots,x_n]^m$ into a square system $AF$ consisting of **$n$ random linear combinations** of these $m$ equations, by multiplying by a random matrix $A\in\mathbb{C}^{n\times m}$. Then it is an easy (but good!) exercise to prove that 
$$\mathbb{V}(F)\subseteq\mathbb{V}(AF).$$ 

The idea is now to solve the square system $AF$, and then manually check which solutions actually correspond to solutions of $F$.

The solutions in the complement $\mathbb{V}(AF)\setminus\mathbb{V}(F)$ are often referred to as **excess solutions**.

Let's try this on the system given in the exericse:
$$\begin{cases}xz - y^2=0\\y - z^2=0\\x - yz=0\\x + y + z + 1=0\,.\end{cases}$$

We begin by picking a random matrix $A\in\mathbb{C}^{3\times 4}$ that we use to obtain the square system.

In [2]:
A = rand(ComplexF64,3,4)

3×4 Matrix{ComplexF64}:
 0.556541+0.837129im   0.174406+0.526931im  …  0.274991+0.0012499im
 0.592238+0.98001im     0.52878+0.455215im     0.359087+0.898572im
 0.255969+0.0773359im  0.771346+0.787903im      0.73441+0.594359im

In [3]:
@var x y z
original_polynomials = [x*z - y^2, y - z^2, x - y*z, x + y + z + 1]

new_polynomials = A * original_polynomials

3-element Vector{Expression}:
 (0.862307462055588 + 0.184646495828475*im)*(x - y*z) + (0.174405942303428 + 0.526930936537934*im)*(y - z^2) + (0.556541190993669 + 0.837129019573366*im)*(x*z - y^2) + (0.274990941857691 + 0.00124990199725716*im)*(1 + x + y + z)
  (0.180878544799798 + 0.0547412174637316*im)*(x - y*z) + (0.528779697207441 + 0.455215389593886*im)*(y - z^2) + (0.592237837450734 + 0.980010082357406*im)*(x*z - y^2) + (0.359087030921866 + 0.898571666419979*im)*(1 + x + y + z)
    (0.241855092010244 + 0.0345666077362697*im)*(x - y*z) + (0.771345652748755 + 0.787902900682577*im)*(y - z^2) + (0.25596929014447 + 0.0773358777619587*im)*(x*z - y^2) + (0.73441010097868 + 0.59435854860755*im)*(1 + x + y + z)

In [4]:
F_new = System(new_polynomials)
res = solve(F_new, start_system = :total_degree)

Tracking 8 paths... 100%|███████████████████████████████| Time: 0:00:04
  # paths tracked:                  8
  # non-singular solutions (real):  5 (1)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         5 (1)


Result with 5 solutions
• 8 paths tracked
• 5 non-singular solutions (1 real)
• random_seed: 0xc16552b2
• start_system: :total_degree


We then evaluate the original polynomials on all the solutions we found, and check if we get something that's almost equal to 0. 

In [5]:
for s in solutions(res)
    value = evaluate(original_polynomials,[x,y,z]=>s)
    println(value)
    println(norm(value))
    #println(norm(F(s)))
    println()
end

ComplexF64[0.0 + 2.44926024096423e-16im, 0.0 + 1.2246634778125917e-16im, -2.5037089277034066e-32 + 0.0im, 1.6678665238184613e-21 + 0.0im]
2.738371114756794e-16

ComplexF64[0.0 + 1.7542598697072487e-16im, 0.0 - 8.771299348536244e-17im, 0.0 + 0.0im, -3.4751686429372884e-17 + 0.0im]
1.9918716284063495e-16

ComplexF64[0.0 + 1.509149956910999e-238im, 0.0 + 2.4492935982947064e-16im, 0.0 - 1.8930723412682805e-238im, 0.0 - 1.509149956910999e-238im]
2.4492935982947064e-16

ComplexF64[4.267345559231888 - 2.398941486997309im, 2.9216000710931516 - 6.475092028411468im, -9.554440893326191 - 3.47687066499722im, -2.53879940436732 + 7.702445756586757im]
15.606949272221698

ComplexF64[-0.042406342263846465 - 0.2594669666611886im, -0.27282626945213606 - 0.26666881794152203im, -0.3514078738888608 + 0.417940429683292im, 0.3417942494531442 + 0.26996792700038885im]
0.8381739220359488



**Conclusion:** Only three of the solutions make sense as solutions of the original system!

### Alternatively: Just use solve directly!

The easy solution is to just feed this to the `solve` command as usual. Then `HomotopyContinuation.jl` will take care of the randomization and checking for you!

In [6]:
@var x y z
original_polynomials = [x*z - y^2, y - z^2, x - y*z, x + y + z + 1]
F = System(original_polynomials)
res = solve(F, start_system = :total_degree)

Tracking 8 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  8
  # non-singular solutions (real):  3 (1)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         3 (1)


Result with 3 solutions
• 8 paths tracked
• 3 non-singular solutions (1 real)
• 2 excess solutions
• random_seed: 0xeda808b8
• start_system: :total_degree


**Note:** Certification won't work for nonsquare systems. This is because it relies on Newton's method, where, again, we invert a Jacobian.

In [7]:
#certify(F,solutions(res))